In [ ]:
!pip install metaphor-python

In [1]:
import spacy
from bs4 import BeautifulSoup

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Function to extract text from HTML
def extract_text_from_html(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    text = ""
    for paragraph in soup.find_all("p"):
        text += paragraph.get_text() + " "
    return text

# Function to generate a shorter search query (limit to 20 words)
def generate_short_search_query(text, max_words=20):
    doc = nlp(text)
    keywords = []

    # Extract nouns and adjectives as keywords
    for token in doc:
        if token.pos_ in ("NOUN", "ADJ"):
            keywords.append(token.text)

    # Limit the keywords to a maximum of max_words
    keywords = keywords[:max_words]

    # Join keywords into a search query
    query = " ".join(keywords)
    return query

def generate_search_query_for_html(html_content):
    text = extract_text_from_html(html_content)
    search_query = generate_short_search_query(text, max_words=20)
    print("Generated Short Search Query (up to 20 words):", search_query)
    return search_query


In [3]:
def get_similar_urls(source_url):
  result = ''
  try:
    response = metaphor_client.find_similar(source_url, num_results = 10)
    matching_results = []
    for res in response.results:
      # print(res.title, res.url)
      result += res.title + " " + res.url + '\n'
    # print(result)
    return (response, result)
  except:
    raise Exception("There seems to be something wrong with the URL you entered, please check")

def get_html_content(response):
  #----------------------------- Finding content of the project -----------------------------
  top_match_url = response.results[0].url # using top match for extracting content of the project
  top_match_id = response.results[0].id
  contents = metaphor_client.get_contents([top_match_id])
  return contents


def find_similar_content(content):
  #-------------------- Generating a search query to retrieve similar content -----------------
  search_query = generate_search_query_for_html(content.contents[0].extract)
  response = metaphor_client.search(
  'Code for ' + search_query,
  num_results=10,
  use_autoprompt=True,
  )
  return (response)



In [4]:
#----------------------------- Initialising Metaphor Client -----------------------------
from metaphor_python import Metaphor
ACCESS_KEY = "b466d1c1-82bb-474e-a6c9-1375ae07ba6c" # ideally this would be fetched from a file
metaphor_client = Metaphor(ACCESS_KEY)

In [ ]:
if __name__ == "__main__":
    report_string = \
    '''\n_________________________________________________________________
      \nPLAGIRAISM REPORT: SIMILAR INFORMATION FOUND ONLINE
      \n__________________________________________________________________
    '''
    source_url = input("Enter URL of Project")
    output_file = "plagiarism_report.txt"
    print("Thanks! Checking for plagiarism - similar content across the web ...")
    response, result = get_similar_urls(source_url)

    report_string += result

    contents = get_html_content(response)
    # print(response)
    response = find_similar_content(contents)

    report_string += "\nOther similar sources: \n"
    for res in response.results:
      report_string += str(res) + "\n"

    print(report_string)
    print("Done! Find your results in : ", output_file)
    with open(output_file, "w", encoding="utf-8") as file:
       file.write(report_string)
